# Sunset on Dakar

This example notebook demonstrates a fairly complex usage scenario of Eradiate. It leverages a scene generator developed as part of the [3DREAMS project](https://www.grasp-earth.com/portfolio/3dreams/) to generate a synthetic scene modelled after an area located close to the city of Dakar, in Senegal. Interested readers can refer to the 3DREAMS project documentation and published material for context and details.

This scene uses a triangulated DEM mesh sourced from the Copernicus GLO-30 DEM dataset, and land cover data from the ESA WorldCover dataset, both resampled at 100 m resolution to keep memory requirements reasonable. The scene is split into three regions:

1. The target region has the highest level of detail and features 100 m-resolution land cover information, and a 30 m-resolution DEM. In this area (a 100 km × 100 km square around the centre of the scene), instances of 3D objects are created on the 100 m × 100 m lattice defined by the DEM and land cover data. The target region is where the simulated instrument is meant to be pointed.

2. The buffer region has a lower level of detail, with the DEM resampled at 100 m-resolution, and features no instances of 3D objects. Its role is to provide realistic diffuse radiation / adjacency to measurements made in the target region.

3. The background region has a uniform diffuse reflectance and should not be radiatively visible in the target region.

When loaded with all possible details (resolved DEM and instances), the scene can be quite large in memory. To allow all users to run this notebook, the default settings remove most detail from the scene. Comments explain how to add more objects and complexity to this lightweight setup.

We start with some basic setup:

In [ ]:
import eradiate
import dreams_scenes as dsc
from pathlib import Path

eradiate.plot.set_style()
eradiate.set_mode("mono")

# Add location of data files to the dreams and mitsuba path resolvers
ROOT_DIR = Path("..").resolve()
DATA_DIR = ROOT_DIR / "data/dreams"

mi_fresolver = dsc.datautil.get_mi_fresolver()

if DATA_DIR not in dsc.fresolver.paths:
    dsc.fresolver.prepend(DATA_DIR)

if str(DATA_DIR) not in mi_fresolver:
    mi_fresolver.prepend(str(DATA_DIR))

Next, we import components that are required to load and render the scene:

In [ ]:
import numpy as np
from eradiate.experiments import AtmosphereExperiment
from eradiate.units import unit_registry as ureg
from eradiate.xarray.interp import dataarray_to_rgb
from matplotlib import pyplot as plt

from dreams_scenes.scene import SceneComponent

To load the scene into memory, we use the 3DREAMS scene loader. The following definition is minimal and focuses on surface features. For now, we will only include the textured surface, without a DEM.

In [ ]:
# Locate scene definition file
DEBUG = (
    True  # If True, use downsampled DEM meshes (less realistic but lower memory usage)
)
scene_id = "Dakar" if not DEBUG else "Dakar_debug"
fname = dsc.fresolver.resolve(f"scenes/{scene_id}.yml", strict=True)

# Configure the scene
scene_config = dsc.scene.SceneConfig.from_yaml(fname)
scene_config.no_topography = True  # Set to False to include the DEM

# Load the scene (should take around 30 s)
kdict, kpmap = scene_config.load_scene(
    components=SceneComponent.SURFACE  # Do not load tiles (lightweight)
    # components=SceneComponent.ALL  # Load tiles (uses more memory)
)

Once the desired surface features are loaded, we configure the measurement, atmosphere and illumination, and we load the scene at the kernel level. That final step will take only a few seconds without tile instances or DEM, and significantly increase (up to several minutes) as the number of features increases. Monitor RAM usage during kernel scene initialization to see if you are putting your hardware under stress. Users with 16 GB of RAM are advised to keep the `DEBUG` flag set to `True`. The highest amount of detail should run comfortably on a setup with 64 GB of RAM.

In [ ]:
# Define measurement
RGB_CHANNELS = [440.0, 550.0, 660.0] * ureg.nm
srf = {"type": "delta", "wavelengths": RGB_CHANNELS}

x = -6.5e3
y = 12.1e3
z = 90.0
origin = [x, y, z] * ureg.m
target = [
    x + np.cos(np.deg2rad(10.0)) * 1000.0,
    y + np.sin(np.deg2rad(15.0)) * 1000.0,
    0.0,
] * ureg.m

camera = [
    {
        "type": "perspective",
        "id": "camera",
        "origin": origin,
        "target": target,
        "up": [0, 0, 1],
        "fov": 35,
        "film_resolution": (854, 480),
        "srf": srf,
        "spp": 64,
    },
]

# Load and initialize experiment
exp = AtmosphereExperiment(
    surface=None,
    atmosphere={"type": "molecular"},
    illumination={"type": "directional", "zenith": 80.0, "azimuth": 290.0},
    measures=camera,
    kdict=kdict,
    kpmap=kpmap,
)
exp.init()

Now, we are ready to render the image. The initial sample count is very low to ensure fast processing on any hardware. Note that the more surface features are added, the more time is spent to compute the intersection between rays and scene geometry.

In [ ]:
# Render image
spp = 4  # Very low, increase to reduce noise
eradiate.run(exp, spp=spp)

In [ ]:
# Display the image
ds = exp.results["camera"]

# Apply basic tone mapping (also scale radiance to make image brighter)
raw_img = dataarray_to_rgb(
    ds["radiance"],
    channels=[("w", 660), ("w", 550), ("w", 440)],
    normalize=False,
)
img = raw_img * 1.8
img = np.clip(img, 0, 1)

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
plt.imshow(img, origin="upper")
ax.set_aspect("equal")
ax.axis("off")
plt.show()
plt.close()

Now that you have rendered this first image, you can do the following:

* Reduce the amount of noise by increasing the number of samples per pixel (`spp`) to 64, or even more if you have a powerful machine and / or time.
* Add buildings and vegetation by setting `components=SceneComponent.ALL` in the scene definition loading cell. This is how the figure in the paper was created.
* Add the DEM by setting `scene_config.no_topography = False` in the same cell. You might notice unnatural angles or tile positions: the scene generator is basic and additional data sanitization would be required to improve realism.
* Increase the resolution of the DEM by setting `DEBUG = False` in the same cell.
